In [1]:
from gad_adversarial_robustness.gad.dominant import dominant
from pygod.utils import load_data
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from sklearn.metrics import roc_auc_score
import os
from gad_adversarial_robustness.utils.graph_utils import prepare_graph
from torch_geometric.data import Data
import numpy as np
from gad_adversarial_robustness.utils.graph_utils import get_n_anomaly_indexes
from typing import List
from gad_adversarial_robustness.poison.greedy import multiple_AS
import copy
from gad_adversarial_robustness.poison.greedy import poison_attack

PRELOADED_EDGE_INDEX = True
EDGE_INDEX_PT = "276_budget_greedy_edge_index.pt"


script_dir = os.path.abspath('')


dataset_caching_path = os.path.join(script_dir, '..', '..', '..', 'data')

In [2]:
clean_data: Data = load_data("inj_cora", dataset_caching_path)
poisoned_data: Data = load_data("inj_cora", dataset_caching_path)

if PRELOADED_EDGE_INDEX is False:
    truth: List[int] = poisoned_data.y.bool()
    amount_of_nodes = poisoned_data.num_nodes
    _, adj, _ = prepare_graph(poisoned_data)
    dense_adj = adj.to_dense()  #Fill in zeroes where there are no edges


    print("Create poison compatible adjacency matrix...")
    triple = []
    for i in range(amount_of_nodes):
        for j in range(i + 1, amount_of_nodes):
            triple.append([i, j, dense_adj[i,j]])  #Fill with 0, then insert actual after



    triple = np.array(triple)

    # These are the nodes we try reduce the active subnetwork score for (disguising anonomaly nodes)
    target_node_lst = get_n_anomaly_indexes(truth, 69)
    #target_node_lst = get_n_anomaly_indexes(truth, 10)
    print(type(target_node_lst)), print(f'target node list: {target_node_lst}'), print(target_node_lst)

    print("Making model...")
    model = multiple_AS(target_lst = target_node_lst, n_node = amount_of_nodes, device = 'cpu')
    budget = 276  # The amount of edges to change


    print("Starting attack...")

    adj_adversary, _, _ = poison_attack(model, triple, budget)

    print("Converting to compatible tensor...")

    # Create Edge Index'
    edge_index = torch.tensor([[],[]])

    # Transpose it to make shape compatible
    transposed_adj_adversary = torch.transpose(adj_adversary, 0, 1)

    for i in range(len(adj_adversary)):
        if(adj_adversary[i][2] != 0):   #If edge value is not 0 (no edge)
            #Add edge to edge index, choosing first 2 elements (edges), and then the ith edge
            edge_index = torch.cat((edge_index, transposed_adj_adversary[:2, i:i+1]), -1)
            # Dataset uses edges both ways so add reverse edge as well
            edge_index = torch.cat((edge_index, torch.flip(transposed_adj_adversary[:2, i:i+1], dims=[0])), -1)


    edge_index = edge_index.type(torch.int64)
    poisoned_data.edge_index = edge_index

else:
    poisoned_data.edge_index = torch.load(EDGE_INDEX_PT)


In [3]:
torch.save(poisoned_data.edge_index, "half_276_budget_greedy_edge_index.pt")

In [4]:
import yaml
#from gad_adversarial_robustness.gad.dominant.dominant import Dominant
import scipy.sparse as sp
from gad_adversarial_robustness.gad.dominant.dominant_cuda import Dominant 
from gad_adversarial_robustness.gad.dominant.dominant_neigh import Dominant as DominantNeigh
from gad_adversarial_robustness.utils.graph_utils import load_anomaly_detection_dataset
from torch_geometric.utils import to_torch_sparse_tensor
yaml_path = os.path.join(script_dir, '..', 'configs', 'dominant_config.yaml')
with open(yaml_path) as file:
        config = yaml.safe_load(file)

# DominantDiffusion on poisoned data


print("DOMINANT  on CLEAN data:")
dataset: Data = load_data("inj_cora")
adj, _, _, adj_label = load_anomaly_detection_dataset(clean_data, config['model']['device'])
#edge_index = torch.LongTensor(np.array(sp.coo_matrix(adj).nonzero()))
adj_label = torch.FloatTensor(adj_label).to(config['model']['device'])
#attrs = torch.FloatTensor(attrs)

edge_index = dataset.edge_index.to(config['model']['device'])
label = torch.Tensor(dataset.y.bool()).to(config['model']['device'])
attrs = dataset.x.to(config['model']['device'])


sparse_adj = to_torch_sparse_tensor(edge_index)



model = DominantNeigh(feat_size=attrs.size(1), hidden_size=config['model']['hidden_dim'], dropout=config['model']['dropout'],
                device=config['model']['device'], edge_index=sparse_adj, adj_label=adj_label, attrs=attrs, label=label)
model.to(config['model']['device'])
model.fit(config, verbose=False)

print("DOMINANT  on POISONED data (greedy w/ 300 budget):")

dataset: Data = load_data("inj_cora")
adj, _, _, adj_label = load_anomaly_detection_dataset(poisoned_data, config['model']['device'])
#edge_index = torch.LongTensor(np.array(sp.coo_matrix(adj).nonzero()))
adj_label = torch.FloatTensor(adj_label).to(config['model']['device'])
#attrs = torch.FloatTensor(attrs)

edge_index = poisoned_data.edge_index.to(config['model']['device'])
label = torch.Tensor(poisoned_data.y.bool()).to(config['model']['device'])
attrs = poisoned_data.x.to(config['model']['device'])


sparse_adj = to_torch_sparse_tensor(edge_index)



model = DominantNeigh(feat_size=attrs.size(1), hidden_size=config['model']['hidden_dim'], dropout=config['model']['dropout'],
                device=config['model']['device'], edge_index=sparse_adj, adj_label=adj_label, attrs=attrs, label=label)
model.to(config['model']['device'])
model.fit(config, verbose=False)

DOMINANT  on CLEAN data:
Epoch: 0119, Auc: 0.8147155021710935
DOMINANT  on POISONED data (greedy w/ 300 budget):
Epoch: 0119, Auc: 0.6191309987029832


In [7]:

print(config['model']['epochs'])

"""
Dominant Aggr on poisoned data
"""

print("DOMINANT modified w/ new TAGConv on CLEAN data:")
adj, _, _, adj_label = load_anomaly_detection_dataset(clean_data, config['model']['device'])
#edge_index = torch.LongTensor(np.array(sp.coo_matrix(adj).nonzero()))
adj_label = torch.FloatTensor(adj_label).to(config['model']['device'])
#attrs = torch.FloatTensor(attrs)

edge_index = clean_data.edge_index.to(config['model']['device'])
label = torch.Tensor(clean_data.y.bool()).to(config['model']['device'])
attrs = clean_data.x.to(config['model']['device'])

jaccard_params= {
    "threshold": 0.01
}

model = Dominant(feat_size=attrs.size(1), hidden_size=config['model']['hidden_dim'], dropout=config['model']['dropout'],
                    device=config['model']['device'], edge_index=edge_index, adj_label=adj_label, attrs=attrs, label=label)
model.to(config['model']['device'])
model.fit(config, verbose=False)

print("DOMINANT modified w/ TAGConv on POISONED data (greedy w/ 300 budget):")

adj, _, _, adj_label = load_anomaly_detection_dataset(poisoned_data, config['model']['device'])
#edge_index = torch.LongTensor(np.array(sp.coo_matrix(adj).nonzero()))
adj_label = torch.FloatTensor(adj_label).to(config['model']['device'])
#attrs = torch.FloatTensor(attrs)

edge_index = poisoned_data.edge_index.to(config['model']['device'])
label = torch.Tensor(poisoned_data.y.bool()).to(config['model']['device'])
attrs = poisoned_data.x.to(config['model']['device'])

jaccard_params= {
    "threshold": 0.01
}

model = Dominant(feat_size=attrs.size(1), hidden_size=config['model']['hidden_dim'], dropout=config['model']['dropout'],
                    device=config['model']['device'], edge_index=edge_index, adj_label=adj_label, attrs=attrs, label=label)
model.to(config['model']['device'])
model.fit(config, verbose=False)




DOMINANT modified w/ new TAGConv on CLEAN data:
Epoch: 0119, Auc: 0.8195172841594767
DOMINANT modified w/ TAGConv on POISONED data (greedy w/ 300 budget):
Epoch: 0000, train_loss=0.81185, train/struct_loss=2.10209, train/feat_loss=0.25889
Epoch: 0000, Auc: 0.6099475554051768
Epoch: 0001, train_loss=0.89915, train/struct_loss=2.29767, train/feat_loss=0.29979
Epoch: 0002, train_loss=0.80871, train/struct_loss=2.10009, train/feat_loss=0.25526
Epoch: 0003, train_loss=0.80867, train/struct_loss=2.10108, train/feat_loss=0.25478
Epoch: 0004, train_loss=0.80848, train/struct_loss=2.10133, train/feat_loss=0.25440
Epoch: 0005, train_loss=0.80832, train/struct_loss=2.10094, train/feat_loss=0.25434
Epoch: 0006, train_loss=0.80837, train/struct_loss=2.10098, train/feat_loss=0.25439
Epoch: 0007, train_loss=0.80842, train/struct_loss=2.10098, train/feat_loss=0.25446
Epoch: 0008, train_loss=0.80834, train/struct_loss=2.10083, train/feat_loss=0.25442
Epoch: 0009, train_loss=0.80826, train/struct_loss=2

In [6]:
"""
import yaml
#from gad_adversarial_robustness.gad.dominant.dominant import Dominant

from gad_adversarial_robustness.gad.dominant.dominant import Dominant 
from gad_adversarial_robustness.gad.dominant.dominant_diffusion import Dominant as DominantDiff
from gad_adversarial_robustness.utils.graph_utils import load_anomaly_detection_dataset

yaml_path = os.path.join(script_dir, '..', 'configs', 'dominant_config.yaml')
with open(yaml_path) as file:
        config = yaml.safe_load(file)

#DominantDiffusion on poisoned data


print("Classic DOMINANT on CLEAN data:")
adj, attrs, label, adj_label = load_anomaly_detection_dataset(clean_data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)

model = Dominant(
        feat_size=attrs.size(1),
        hidden_size=config["model"]["hidden_dim"],
        dropout=config["model"]["dropout"],
        device=config["model"]["device"],
        adj=adj,
        adj_label=adj_label,
        attrs=attrs,
        label=label,
    )
model.fit(config)

print("Classic DOMINANT on POISONED data (greedy w/ 300 budget):")
adj, attrs, label, adj_label = load_anomaly_detection_dataset(poisoned_data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)

model = Dominant(
        feat_size=attrs.size(1),
        hidden_size=config["model"]["hidden_dim"],
        dropout=config["model"]["dropout"],
        device=config["model"]["device"],
        adj=adj,
        adj_label=adj_label,
        attrs=attrs,
        label=label,
    )
model.fit(config)



#DominantDiffusion on poisoned data


print("DOMINANT modified w/ diffusion on CLEAN data:")
adj, attrs, label, adj_label = load_anomaly_detection_dataset(clean_data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)
model = DominantDiff(
        feat_size=attrs.size(1),
        hidden_size=config["model"]["hidden_dim"],
        dropout=config["model"]["dropout"],
        device=config["model"]["device"],
        adj=adj,
        adj_label=adj_label,
        attrs=attrs,
        label=label,
    )

model.fit(config)

print("DOMINANT modified w/ diffusion on POISONED data (greedy w/ 300 budget):")


adj, attrs, label, adj_label = load_anomaly_detection_dataset(poisoned_data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)
model = DominantDiff(
        feat_size=attrs.size(1),
        hidden_size=config["model"]["hidden_dim"],
        dropout=config["model"]["dropout"],
        device=config["model"]["device"],
        adj=adj,
        adj_label=adj_label,
        attrs=attrs,
        label=label,
    )

model.fit(config)

"""

'\nimport yaml\n#from gad_adversarial_robustness.gad.dominant.dominant import Dominant\n\nfrom gad_adversarial_robustness.gad.dominant.dominant import Dominant \nfrom gad_adversarial_robustness.gad.dominant.dominant_diffusion import Dominant as DominantDiff\nfrom gad_adversarial_robustness.utils.graph_utils import load_anomaly_detection_dataset\n\nyaml_path = os.path.join(script_dir, \'..\', \'configs\', \'dominant_config.yaml\')\nwith open(yaml_path) as file:\n        config = yaml.safe_load(file)\n\n#DominantDiffusion on poisoned data\n\n\nprint("Classic DOMINANT on CLEAN data:")\nadj, attrs, label, adj_label = load_anomaly_detection_dataset(clean_data)\nadj = torch.FloatTensor(adj)\nadj_label = torch.FloatTensor(adj_label)\nattrs = torch.FloatTensor(attrs)\n\nmodel = Dominant(\n        feat_size=attrs.size(1),\n        hidden_size=config["model"]["hidden_dim"],\n        dropout=config["model"]["dropout"],\n        device=config["model"]["device"],\n        adj=adj,\n        adj_labe